<a href="https://colab.research.google.com/github/outinletter/DataAnalysis/blob/main/%EC%BD%94%EB%94%A9%2C_text_gen_gpt2%2C_google_translator%2C_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사진에서 객체 추출

# transformers GPT2 설치

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 59 kB 8.9 MB/s 
     |████████████████████████████████| 895 kB 94.8 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
     |████████████████████████████████| 3.3 MB 88.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import transformers
import torch

In [3]:
gpt_tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-large')
gpt_model = transformers.GPT2LMHeadModel.from_pretrained('gpt2-large')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

In [4]:
def gen_text(prompt_text, tokenizer, model, n_seqs=1, max_length=25):
  # n_seqs is the number of sentences to generate

  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=max_length+len(encoded_prompt),
      # so we add the length of the original sequence to max_length
      temperature=1.0,
      top_k=0,
      top_p=0.9,
      repetition_penalty=1.2, # To ensure that we dont get repeated phrases
      do_sample=True,
      num_return_sequences=n_seqs
  )

  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # the _ indicates that the operation will be done in-place


  generated_sequences = []
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence)
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True, )) :]
    )
    generated_sequences.append(total_sequence)
  return generated_sequences

In [5]:
## One important note: in our function, on line 5, make sure that
# return_tensor is return_tensors, otherwise you will get an error like
# this:
#####
# Another important note: on line 27 of the function, instead of
# clear_up_tokenization_spaces, write clean_up_tokenization_spaces
####

In [6]:
# gen_text("I have a dream that one day down in Alabama, with its vicious racists,",
#          gpt_tokenizer,
#          gpt_model,
#          max_length=40,
#          n_seqs=3)

In [7]:
results=gen_text("wow, beer, rock-n-roll music, beach", gpt_tokenizer, gpt_model, max_length=100, n_seqs=3)
results

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['wow, beer, rock-n-roll music, beach parties - New York had them all."\n\nBreyer went on to see tours from his late Father Ernest Belmont who produced national hits for Jimmy James and Poco.\n\nFive years ago Bill Yorn opened the Sands Bethlehem set called Light & Sound with Irish of origin Robbie O\'Connell while John Ford released a side on the band Wigwam in 2003 called Three (cancelled) Ladies of the Moodhen Stones',
 'wow, beer, rock-n-roll music, beach weather, orange sherbet — that\'s what every one of us wants, a table for dining at.\n\nLet\'s see…what else is our room answer? "Hollywood" wants an astroturfed picnic table but no burrito bar on this floor (was it ever simple?). Did we mention that the men always have to be up at 3am to catch Barbara Walters while you wait in line for dinner? That did',
 'wow, beer, rock-n-roll music, beach music – it wasn\'t until I reached the far end of the road that we discovered a village – and that was almost as much fun as any hotel I\'d 

# 구글 번역 googletrans

In [8]:
#!pip uninstall googletrans
#!pip install googletrans

In [9]:
!pip install googletrans==4.0.0-rc1 #> /dev/null

     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 1.3 MB 10.4 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=2c79c55d7cd39a96f7f229d5edbcd489cae4546ed176b4a6801beee16b71db3c
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [10]:
from googletrans import Translator
translator = Translator()
result = translator.translate(results[0], dest="ko")
print(result.text)

와우, 맥주, 록 - 롤 음악, 해변 파티 - 뉴욕은 모두 그들을 모두 가지고있었습니다. "

Breyer는 Jimmy James와 Poco를 위해 국가 히트를 생산 한 늦은 아버지의 맨스트리트 벨몬트에서 투어를 보러갔습니다.

5 년 전 Bill Yorn은 Origin Robbie O'Connell의 아일랜드의 아일랜드와 함께 빛과 소리라고 불리는 샌즈 베들레헴 세트를 열었습니다.


In [11]:
from googletrans import Translator

translator = Translator()

en_list = []
error_list = []

for i in range(len(results)):
    try:
        title_list = results[i]
        temp_en = translator.translate(title_list, dest='ko').text 
                # translator.translate(str1, str="ko", dest='en')
        en_list.append(temp_en)
    except:
        error_list.append(result)
        pass
en_list

['와우, 맥주, 록 - 롤 음악, 해변 파티 - 뉴욕은 모두 그들을 모두 가지고있었습니다. "\n\nBreyer는 Jimmy James와 Poco를 위해 국가 히트를 생산 한 늦은 아버지의 맨스트리트 벨몬트에서 투어를 보러갔습니다.\n\n5 년 전 Bill Yorn은 Origin Robbie O\'Connell의 아일랜드의 아일랜드와 함께 빛과 소리라고 불리는 샌즈 베들레헴 세트를 열었습니다.',
 '와우, 맥주, 락앤롤 음악, 해변 날씨, 오렌지 셔벗 - 우리 모두가 원하는 모든 것이 원하는 테이블입니다.\n\n보자. 우리 방 대답은 무엇입니까?"할리우드"는 Astroturfed Picnic 테이블을 원하지만이 바닥에 Burrito Bar가 없었습니다 (그것은 단순 했습니까?).우리는 저녁 식사를 위해 라인에서 기다리는 동안 남성이 항상 3am에 항상 Barbara Walters를 잡아야 할 것입니다.그랬어',
 '와우, 맥주, 록웍 앤 롤 음악, 해변 음악 - 우리가 마을을 발견 한 도로의 맨 끝까지 도착 할 때까지는 아니 었습니다.. "\n\nKembo의 경로 - 전 동료와 레코드 회사가 독일, 스페인 및 다른 곳에서 흩어져 (Ex-Slim Shoes Drummer STIG의 많은 사람들) - 필연적으로 자란 차이가 있습니다.그의 국제 집 밴드 명성에 대한 그의 유도는 마찬가지로']

# Creating the interface and launching

In [12]:
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 979 kB 5.3 MB/s 
     |████████████████████████████████| 206 kB 83.0 MB/s 
     |████████████████████████████████| 1.9 MB 49.6 MB/s 
     |████████████████████████████████| 63 kB 2.9 MB/s 
     |████████████████████████████████| 961 kB 69.5 MB/s 
     |████████████████████████████████| 3.6 MB 30.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [13]:
import gradio as gr
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

### Loading the model and creating the generate function

Note: You can also change `gpt2` to `gpt2-xl` for a much more powerful model!

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [15]:
def generate_text(inp):
    input_ids = tokenizer.encode(inp, return_tensors='tf')
    beam_output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return ".".join(output.split(".")[:-1]) + "."

In [16]:
output_text = gr.outputs.Textbox()
gr.Interface(generate_text,"textbox", output_text, title="GPT-2",
             description="OpenAI's GPT-2 is an unsupervised language model that \
             can generate coherent text. Go ahead and input a sentence and see what it completes \
             it with! Takes around 20s to run.").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17916.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://17916.gradio.app')



```
# 코드로 형식 지정됨
```

how did the universe come into existence?
are machine intelligent than human being?

# 한글 맞춤법

In [17]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-9a5f3itr
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-9a5f3itr
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=e18c21c1f142432002f0033af68f3a901c562939a76dca9254838c4058e8f365
  Stored in directory: /tmp/pip-ephem-wheel-cache-idfk321h/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker
